In [1]:
#パッケージのダウンロード
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time 
from urllib.request import urlopen
import numpy as np
from urllib.parse import urljoin
import MySQLdb

In [3]:
#Mysqlコネクション
conn = MySQLdb.connect(host = ,user = , passwd = ,charset = 'utf8mb4',db = )
#カーソルの取得
c = conn.cursor()

In [4]:
#ベースとなるURL
base_url = 'https://www.etudehouse.com/jp/ja/products/list?pageno='
#商品名格納用
name = []
#URL格納用
url = []
#価格格納用
price = []
#商品紹介格納用
intro = []
#ジャンル格納用
genre = []

#
for i in range(1,6):
    #トップページの取得
    main_url = base_url + str(i)
    res = requests.get(main_url)
    #商品ごとのアクセス先の取得
    soup = BeautifulSoup(res.text, 'html.parser')
    items = soup.find_all('p', class_='item-name')
    for j in items:
        #name.append(j.find('a').text)
        #商品ごとのアクセセス先にアクセスする
        url_intro = j.find('a').get('href')
        url.append(url_intro)
        res_2 = requests.get(url_intro)
        soup_2 =  BeautifulSoup(res_2.text, 'html.parser')
        
        #商品名
        tmp = soup_2.find_all('p', class_='item-ttl clone-item')
        if tmp != []:
            name.append(tmp[0].get_text())
        else:
            name.append('')
            
        #価格
        tmp = soup_2.find_all('p', class_='item-price-main')
        if tmp != []:
            tmp = tmp[0].get_text()
            if re.search('^¥(\s[1-9]\d*|\s[1-9]\d{0,2}(,\d{3})+)\n',tmp) is None:
                tmp = soup_2.find_all('p', class_='item-price-sub')
                tmp = tmp[0].get_text()
                tmp = re.search('^¥(\s[1-9]\d*|\s[1-9]\d{0,2}(,\d{3})+)\n',tmp).group(0)
                price.append(tmp.replace('¥','').replace('\n','').replace(',','').replace(' ',''))
            else:
                tmp =re.search('^¥(\s[1-9]\d*|\s[1-9]\d{0,2}(,\d{3})+)\n',tmp).group(0)
                price.append(tmp.replace('¥','').replace('\n','').replace(',','').replace(' ',''))
        else:
            price.append('')
            
        #イントロ
        tmp = soup_2.find_all('p', class_='item-desc')
        if tmp != []:
            intro.append(tmp[0].get_text(','))
        else:
            intro.append('')
        
        #ジャンル
        tmp_a = soup_2.find_all('ul', class_='breadcrumb')
        if tmp_a != []:
            tmp = tmp_a[0].get_text().split('\n')
            if tmp[len(tmp)-3] == '':
                genre.append(tmp[len(tmp)-4])
            elif tmp[len(tmp)-2] == '':
                genre.append(tmp[len(tmp)-3])
            else:
                genre.append(tmp[len(tmp)-2])
            if genre[len(genre)-1] == '':
                tmp = tmp_a[1].get_text().split('\n')
                if tmp[len(tmp)-3] == '':
                    genre[len(genre)-1] = tmp[len(tmp)-4]
                elif tmp[len(tmp)-2] == '':
                    genre[len(genre)-1] = tmp[len(tmp)-3]
                else:
                    genre[len(genre)-1] = tmp[len(tmp)-2]
        else:
            genre.append('')

#データセットの作成           
etude_product = pd.DataFrame(
    {'name': name,
     'url': url,
     'price': price,
     'intro': intro,
     'genre': genre,
                           }
)            

#被っているアイテムを落とす
etude_product = etude_product.drop_duplicates(subset='url')

#落とした時にバラバラになるindexを修正
etude_product =  etude_product.reset_index(drop=True)
            
            
#Mysqlコネクション
conn = MySQLdb.connect(host = 'localhost',user = 'root', passwd = '4m@@hWenxCmD@d6',charset = 'utf8mb4',db = 'ex_scraping')

#カーソルの取得
c = conn.cursor()
#テーブルの定義、作成
c.execute('DROP TABLE IF EXISTS `etude`')
c.execute("""
    CREATE TABLE `etude`(
        `id` int(11) NOT NULL AUTO_INCREMENT,
        `name` varchar(255),
        `url` varchar(255),
        `price` int,
        `intro` varchar(255),
        `genre` varchar(255),
        PRIMARY KEY (id)
        )
""")
#データベースの変更をコミット
conn.commit()

#nanを原因とするエラーの防止
etude_product = etude_product.fillna('NULL')

#データベースへの書き込み
for i in range(len(etude_product['name'])):
    c.execute('INSERT INTO `etude`(name,url,price,intro,genre) VALUES (%s, %s, %s, %s, %s)',
               (etude_product['name'][i],etude_product['url'][i],etude_product['price'][i],etude_product['intro'][i],etude_product['genre'][i]))

#データベースの変更をコミット
conn.commit()
